# Advanced R&D with PowerModelsONM

**Author**:\
David M Fobes\
Deputy Group Leader\
A-1 Information Systems and Modeling Group\
[dfobes@lanl.gov](mailto:dfobes@lanl.gov)

LA-UR-25-20156

**Los Alamos National Laboratory Grid Science Winter School 2025**

This is an advanced introduction to developing an advanced problem formulation with PowerModelsONM and PowerModelsDistribution. The PowerModelsITD and PowerModelsONM tutorials are prerequisities to this notebook.

# Development Environment Setup

It is recommended that you create a new **project** when developing your own custom workflows, formulations, problems, etc.

In [1]:
using Pkg

Pkg.activate("./tmp")

Pkg.add("PowerModelsONM")
Pkg.add("PowerModelsDistribution")
Pkg.add("InfrastructureModels")
Pkg.add("JuMP")

  Activating project at `~/Library/CloudStorage/OneDrive-LosAlamosNationalLaboratory/local/working/winter-school/tmp`
┌ Warning: could not download https://pkg.julialang.org/registries
│   exception = RequestError: Could not resolve host: pkg.julialang.org while requesting https://pkg.julialang.org/registries
└ @ Pkg.Registry ~/.julia/juliaup/julia-1.11.2+0.aarch64.apple.darwin14/share/julia/stdlib/v1.11/Pkg/src/Registry/Registry.jl:77
 registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Library/CloudStorage/OneDrive-LosAlamosNationalLaboratory/local/working/winter-school/tmp/Project.toml`
  No Changes to `~/Library/CloudStorage/OneDrive-LosAlamosNationalLaboratory/local/working/winter-school/tmp/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Library/CloudStorage/OneDrive-LosAlamosNationalLaboratory/local/working/winter-school/tmp/Project.toml`
  No Changes to `~/Library/CloudStorage/OneDrive-LosAlamosNationalLaboratory

The reason you might instead clone the PowerModelsONM repository and activate that project in Julia is when you plan to develop the ONM package itself, e.g., to submit a bugfix, new feature, etc. 

Alternatively, you might do this to debug an issue you are seeing, but we also recommend a slightly different workflow for this:

To setup the folder structure, in your terminal:

```bash
cd working

git clone https://github.com/lanl-ansi/PowerModelsONM.jl

mkdir myproject && cd myproject

julia --project=.
```

In your Julia REPL:

```julia
]dev ../PowerModelsONM.jl
```

or 

```julia
using Pkg
Pkg.develop("../PowerModelsONM.jl")
```

## Tutorial Setup

Here we do our standard imports setup paths and create a solver.

We recommend using `import ... as ..` or just `import` so that it is clear from your code where different function originate.

In this tutorial we will need `JuMP` explicitly, to create new variables, constraints, objectives, etc.

Since we will only use the MILP formulation (i.e., `LinDist3Flow`), `HiGHS` will be an appropriate solver for the whole notebook.

In [2]:
import PowerModelsONM as ONM
import PowerModelsDistribution as PMD
import InfrastructureModels as IM
import JuMP

ieee13_network_path = joinpath(dirname(pathof(ONM)), "../examples/data/network.ieee13mod.dss")
ieee13_settings_path = joinpath(dirname(pathof(ONM)), "../examples/data/settings.ieee13mod.json")

solver = ONM.optimizer_with_attributes(
    ONM.HiGHS.Optimizer,
    "presolve" => "off",
    "primal_feasibility_tolerance" => 1e-6,
    "dual_feasibility_tolerance" => 1e-6,
    "small_matrix_value" => 1e-12,
    "allow_unbounded_or_infeasible" => true,
    "output_flag" => false
)

MathOptInterface.OptimizerWithAttributes(HiGHS.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("presolve") => "off", MathOptInterface.RawOptimizerAttribute("primal_feasibility_tolerance") => 1.0e-6, MathOptInterface.RawOptimizerAttribute("dual_feasibility_tolerance") => 1.0e-6, MathOptInterface.RawOptimizerAttribute("small_matrix_value") => 1.0e-12, MathOptInterface.RawOptimizerAttribute("allow_unbounded_or_infeasible") => true, MathOptInterface.RawOptimizerAttribute("output_flag") => false])

# The Data Models in PowerModelsDistribution

The Data Model portion of this notebook is the only section that really is unique to `PowerModelsDistribution` derived packages like `PowerModelsONM`. Because PowerModelsDistribution has two data models, the ENGINEERING model and the MATHEMATICAL model, additional work is needed to get data to pass through that transformation.

In this notebook we use the convention `eng` for `ENGINEERING` data, and `math` for `MATHEMATICAL` data. Also, `_s` indicates some settings have been applied, and `mn_` indicates a multinetwork data structure.

First we import our base network data.

In [3]:
eng = ONM.parse_file(ieee13_network_path)

[ PowerModelsDistribution | Info ] : Circuit has been reset with the 'clear' on line 1 in 'network.ieee13mod.dss'
[ PowerModelsDistribution | Info ] : Command 'calcv' on line 166 in 'network.ieee13mod.dss' is not supported, skipping.
[ PowerModelsDistribution | Info ] : Command 'solve' on line 180 in 'network.ieee13mod.dss' is not supported, skipping.


Dict{String, Any} with 21 entries:
  "recloser"                => Dict{String, Any}("671700"=>Dict{String, Any}("l…
  "is_kron_reduced"         => true
  "conductor_ids"           => [1, 2, 3]
  "bus"                     => Dict{String, Any}("800aux"=>Dict{String, Any}("r…
  "name"                    => "ieee13nodeckt"
  "settings"                => Dict{String, Any}("sbase_default"=>1000.0, "vbas…
  "files"                   => ["/Users/dfobes/.julia/packages/PowerModelsONM/T…
  "switch_close_actions_ub" => Inf
  "storage"                 => Dict{String, Any}("battery_mg1a"=>Dict{String, A…
  "switch"                  => Dict{String, Any}("cb_101"=>Dict{String, Any}("c…
  "generator"               => Dict{String, Any}("675"=>Dict{String, Any}("vg"=…
  "dss_options"             => DssOptions("%growth"=>2.5, "%mean"=>65.0, "%norm…
  "time_series"             => Dict{String, Any}("microgrid1a"=>Dict{String, An…
  "voltage_source"          => Dict{String, Any}("source"=>Dict{String, Any}(

Next we generate some basic settings

In [4]:
settings = ONM.build_settings_new(
    eng;
    vm_lb_pu=0.9,
    vm_ub_pu=1.1,
    line_limit_multiplier=Inf,
    transformer_limit_multiplier=Inf,
    vad_deg=10.0,
    timestep_hours=0.10,
    switch_close_actions_ub=1
)

Dict{String, Any} with 5 entries:
  "options" => Dict{String, Any}("outputs"=>Dict{String, Any}("log-level"=>"war…
  "line"    => Dict{String, Any}("632670"=>Dict{String, Any}("vad_ub"=>[10.0, 1…
  "solvers" => Dict{String, Any}("HiGHS"=>Dict{String, Any}("presolve"=>"choose…
  "switch"  => Dict{String, Any}("cb_101"=>Dict{String, Any}("cm_ub"=>[Inf, Inf…
  "bus"     => Dict{String, Any}("646"=>Dict{String, Any}("vm_ub"=>[2.64195, 2.…

In [7]:
PMD.apply_voltage_bounds!(eng; vm_lb=0.9, vm_ub=1.1)

and apply them

In [8]:
eng_s = ONM.apply_settings(eng, settings)

Dict{String, Any} with 24 entries:
  "recloser"                => Dict{String, Any}("671700"=>Dict{String, Any}("l…
  "is_kron_reduced"         => true
  "conductor_ids"           => [1, 2, 3]
  "time_elapsed"            => [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
  "bus"                     => Dict{String, Any}("800"=>Dict{String, Any}("rg"=…
  "name"                    => "ieee13nodeckt"
  "solvers"                 => Dict{String, Any}("HiGHS"=>Dict{String, Any}("pr…
  "settings"                => Dict{String, Any}("sbase_default"=>1000.0, "vbas…
  "files"                   => ["/Users/dfobes/.julia/packages/PowerModelsONM/T…
  "switch_close_actions_ub" => [1, 1, 1, 1, 1, 1, 1, 1]
  "storage"                 => Dict{String, Any}("battery_mg1a"=>Dict{String, A…
  "switch"                  => Dict{String, Any}("801675"=>Dict{String, Any}("c…
  "generator"               => Dict{String, Any}("675"=>Dict{String, Any}("pg"=…
  "dss_options"             => DssOptions("%growth"=>2.5, "%mean"

Because the base workflow in ONM is centered around multinetwork problems, even when working with the base network there can be an issue with applying settings where `switch_close_actions_ub` and `time_elapsed` are vectors in the non-multinetwork structure.

In [9]:
# Fix vector properties
eng_s["switch_close_actions_ub"]=eng_s["switch_close_actions_ub"][1]
eng_s["time_elapsed"]=eng_s["time_elapsed"][1]

0.1

# Adding Fields to Data Models

Lets say that you have some custom data you want to add to a particular generator in the network.

First, let's look at the properties we have parsed from OpenDSS.

In [10]:
eng_s["generator"]["675"]

Dict{String, Any} with 15 entries:
  "pg"            => [166.667, 166.667, 166.667]
  "connections"   => [1, 2, 3]
  "bus"           => "675"
  "configuration" => WYE
  "qg"            => [116.667, 116.667, 116.667]
  "status"        => ENABLED
  "dss"           => Dict{String, Any}("kva"=>"500", "name"=>"675", "phases"=>"…
  "vg"            => [1.38564, 1.38564, 1.38564]
  "qg_ub"         => [233.333, 233.333, 233.333]
  "phases"        => 3
  "source_id"     => "generator.675"
  "qg_lb"         => [-233.333, -233.333, -233.333]
  "pg_lb"         => [0.0, 0.0, 0.0]
  "pg_ub"         => [166.667, 166.667, 166.667]
  "control_mode"  => FREQUENCYDROOP

Now we can add a new property `ramp_rate`, which is in units % of rated capacity per minute.

In [11]:
eng_s["generator"]["675"]["ramp_rate"] = 3.0 # % of rated capacity / min


3.0

If we transform the data model to `MATHEMATICAL`, we notice that this new property is missing.

In [13]:
math = ONM.transform_data_model(eng_s)

math["gen"]["1"]["ramp_rate"]

LoadError: KeyError: key "ramp_rate" not found

To get this new property for when we are working with the JuMP Model, we need to use the passthrough interface.

In [14]:
my_eng2math_passthrough = Dict{String,Vector{String}}(
    "generator"=>String["ramp_rate"]
)

Dict{String, Vector{String}} with 1 entry:
  "generator" => ["ramp_rate"]

In [15]:
math = ONM.transform_data_model(eng_s, eng2math_passthrough=my_eng2math_passthrough)

math["gen"]["1"]

Dict{String, Any} with 22 entries:
  "pg"            => [0.166667, 0.166667, 0.166667]
  "model"         => 2
  "connections"   => [1, 2, 3]
  "shutdown"      => 0.0
  "startup"       => 0.0
  "configuration" => WYE
  "name"          => "675"
  "qg"            => [0.116667, 0.116667, 0.116667]
  "gen_bus"       => 8
  "pmax"          => [0.166667, 0.166667, 0.166667]
  "dss"           => Dict{String, Any}("kva"=>"500", "name"=>"675", "phases"=>"…
  "vbase"         => 2.40178
  "source_id"     => "generator.675"
  "index"         => 1
  "cost"          => [1.0, 0.0]
  "gen_status"    => 1
  "ramp_rate"     => 3.0
  "qmin"          => [-0.233333, -0.233333, -0.233333]
  "qmax"          => [0.233333, 0.233333, 0.233333]
  "control_mode"  => 0
  "pmin"          => [0.0, 0.0, 0.0]
  "ncost"         => 2

# AbstractUnbalancedPowerModel

When building a problem, we are working with a Julia Struct called the AbstractUnbalancedPowerModel. The will be different for other packages, but the structure and helper function all work the same way.

First, lets transform our data to multinetwork.

In [16]:
mn_eng_s = ONM.make_multinetwork(eng_s)

Dict{String, Any} with 9 entries:
  "options"      => Dict{String, Any}("outputs"=>Dict{String, Any}("log-level"=…
  "name"         => "ieee13nodeckt"
  "solvers"      => Dict{String, Any}("HiGHS"=>Dict{String, Any}("presolve"=>"c…
  "files"        => ["/Users/dfobes/.julia/packages/PowerModelsONM/ThIPC/src/..…
  "multinetwork" => true
  "dss_options"  => DssOptions("%growth"=>2.5, "%mean"=>65.0, "%normal"=>100.0,…
  "nw"           => Dict{String, Any}("3"=>Dict{String, Any}("recloser"=>Dict{S…
  "mn_lookup"    => Dict("8"=>0.7, "4"=>0.3, "1"=>0.0, "5"=>0.4, "2"=>0.1, "6"=…
  "data_model"   => ENGINEERING

Next, let's create the AbstractUnbalancedPowerModel. To do so, you need at a minimum

- input data
- formulation
- problem definition

In [18]:
pm = ONM.instantiate_onm_model(mn_eng_s, ONM.LPUBFDiagPowerModel, ONM.build_mn_block_mld; eng2math_passthrough=my_eng2math_passthrough, multinetwork=true);

The primary properties of this struct are the following `Symbol`s

In [19]:
propertynames(pm)

(:model, :data, :setting, :solution, :ref, :var, :con, :sol, :sol_proc, :ext)

These are the key properties you will care about, which are consistent across the different packages in their usage.

- `:model`: JuMP Model
- `:data`: The `MATHEMATICAL` data model used in the creation of the `:model`
- `:solution`: The solution, which will not be populated until `solve` is called
- `:ref`: a reference dict used to quickly access different aspects of the data
- `:var`: a quick reference to the different variables created
- `:con`: a quick reference to the different constraints created
- `:sol`: a way to specify which variables / expressions should appear in the solution

There are a few useful helper functions used to access different parts of the struct. 

To access the subnetworks of the multinetwork, use `nws`

In [20]:
ONM.nws(pm)

Dict{Int64, Any} with 8 entries:
  5 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  4 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  6 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  7 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  2 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  8 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  3 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…
  1 => Dict{Symbol, Any}(:block_line_losses=>Dict(5=>0.0, 4=>0.0651414, 6=>0.05…

Or if you just want the multinetwork ids, `nw_ids`

In [21]:
ONM.nw_ids(pm)

KeySet for a Dict{Int64, Any} with 8 entries. Keys:
  5
  4
  6
  7
  2
  8
  3
  1

`ref` allows us to access special aspects of the data. These are more than just the data itself, although that is usually contained as well. The things in this dictionary will be different for each package, dependent on the types of problems trying to be solved. It is possible to extend the `ref` with an extension to add custom data, but that is out of scope here. See PowerModelsONM source for an example where we extend the PowerModelsDistribution `ref`.

In [22]:
ONM.ref(pm, 1)

Dict{Symbol, Any} with 70 entries:
  :block_line_losses       => Dict(5=>0.0, 4=>0.0651414, 6=>0.0579035, 7=>5.760…
  :buspairs                => Dict{NTuple{4, Int64}, Dict{String, Real}}((23, 8…
  :block_graph             => SimpleGraph{Int64}(6, [[3, 6, 7], [4, 5], [1], [2…
  :substation_blocks       => [3]
  :bus_conns_gen           => Dict(5=>[], 35=>[], 20=>[], 16=>[], 12=>[(2, [1, …
  :shunt_block_map         => Dict(2=>1, 1=>7)
  :block_loads             => Dict{Int64, Set}(5=>Set([9]), 4=>Set([7, 14]), 6=…
  :options                 => Dict{String, Any}("data"=>Dict{String, Any}("time…
  :gen                     => Dict{Int64, Any}(4=>Dict{String, Any}("pg"=>[0.0,…
  :switch_dispatchable     => Dict{Int64, Any}(5=>Dict{String, Any}("f_connecti…
  :load                    => Dict{Int64, Any}(5=>Dict{String, Any}("model"=>IM…
  :switch_scores           => Dict(5=>1.64499, 4=>2.30142, 6=>1.64557, 7=>3.090…
  :arcs_branch_to          => [(5, 8, 23), (16, 26, 27), (20, 32, 33), (12

In [23]:
ONM.ref(pm, 1, :gen, 1)

Dict{String, Any} with 22 entries:
  "pg"            => [0.166667, 0.166667, 0.166667]
  "model"         => 2
  "connections"   => [1, 2, 3]
  "shutdown"      => 0.0
  "startup"       => 0.0
  "configuration" => WYE
  "name"          => "675"
  "qg"            => [0.116667, 0.116667, 0.116667]
  "gen_bus"       => 8
  "pmax"          => [0.166667, 0.166667, 0.166667]
  "dss"           => Dict{String, Any}("kva"=>"500", "name"=>"675", "phases"=>"…
  "vbase"         => 2.40178
  "source_id"     => "generator.675"
  "index"         => 1
  "cost"          => [1.0, 0.0]
  "gen_status"    => 1
  "ramp_rate"     => 3.0
  "qmin"          => [-0.233333, -0.233333, -0.233333]
  "qmax"          => [0.233333, 0.233333, 0.233333]
  "control_mode"  => 0
  "pmin"          => [0.0, 0.0, 0.0]
  "ncost"         => 2

Use `ids` if we only want the ids (keys) of the ref dictionaries.

In [24]:
ONM.ids(pm, 1, :gen)

KeySet for a Dict{Int64, Any} with 4 entries. Keys:
  4
  2
  3
  1

`var` is used to access the different JuMP variables.

In [25]:
ONM.var(pm, 1)

Dict{Symbol, Any} with 42 entries:
  :alpha                    => Dict{Tuple{Int64, Int64}, Union{Int64, VariableR…
  :p                        => Dict{Tuple{Int64, Int64, Int64}, DenseAxisArray{…
  :CCdi                     => Dict{Int64, Matrix{AffExpr}}(20=>[0 -1_CCdi_32[2…
  :CCdr                     => Dict{Int64, Symmetric{VariableRef, Matrix{Variab…
  :z_inverter               => Dict{Tuple{Symbol, Int64}, Union{Int64, Variable…
  :se                       => 1-dimensional DenseAxisArray{JuMP.VariableRef,1,…
  :qsw                      => Dict{Any, Any}((1, 14, 11)=>1-dimensional DenseA…
  :ps                       => Dict{Int64, DenseAxisArray{VariableRef, 1, Tuple…
  :lambda                   => Dict{Tuple{Int64, Int64}, VariableRef}((6, 8)=>1…
  :w                        => Dict{Int64, DenseAxisArray{VariableRef, 1, Tuple…
  :qs                       => Dict{Int64, DenseAxisArray{VariableRef, 1, Tuple…
  :capacitor_state          => Dict{Any, Any}()
  :tap                    

In [26]:
ONM.var(pm, 1, :pg, 1)

1-dimensional DenseAxisArray{JuMP.VariableRef,1,...} with index sets:
    Dimension 1, [1, 2, 3]
And data, a 3-element Vector{JuMP.VariableRef}:
 1_pg_1[1]
 1_pg_1[2]
 1_pg_1[3]

# Standard Problem

This is the standard problem that we are modifying in the sections below, for reference

In [27]:
result = ONM.solve_onm_model(mn_eng_s, ONM.LPUBFDiagPowerModel, solver, ONM.build_mn_block_mld; multinetwork=true)

Dict{String, Any} with 9 entries:
  "solve_time"         => 5.49763
  "optimizer"          => "HiGHS"
  "termination_status" => OPTIMAL
  "dual_status"        => NO_SOLUTION
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 5.31939
  "solution"           => Dict{String, Any}("nw"=>Dict{String, Dict{String, Any…
  "mip_gap"            => 0.0
  "objective_lb"       => 5.31939

# Adding a Custom Constraint to Existing Problem

In the following example we add a custom constraint using the `ramp_rate` data we added previously. Here we are constraining the new power output from the generator for which `ramp_rate` was defined to adhere to that rate.

We make the assumption that the generator is off at the initial timestep, and needs to be ramped. In each subsequent timestep, we constrain the power output to only increase by the allowed rate. 

In [28]:
function my_new_start_constraint(pm::ONM.AbstractUnbalancedPowerModel, nw::Int, i::Int)
    gen = ONM.ref(pm, nw, :gen, i)

    if !ismissing(get(gen, "ramp_rate", missing))
        JuMP.@constraint(pm.model, sum(ONM.var(pm, nw, :pg, i)) <= gen["ramp_rate"]/100.0 * sum(gen["pmax"]) * ONM.ref(pm, nw, :time_elapsed) * 60.0)
    end
end

function my_new_constraint(pm::ONM.AbstractUnbalancedPowerModel, nw_1::Int, nw_2::Int, i::Int)
    gen = ONM.ref(pm, nw_2, :gen, i)

    if !ismissing(get(gen, "ramp_rate", missing))
        JuMP.@constraint(pm.model, sum(ONM.var(pm, nw_2, :pg, i)) - sum(ONM.var(pm, nw_1, :pg, i)) <= gen["ramp_rate"]/100.0 * sum(gen["pmax"]) * ONM.ref(pm, nw_2, :time_elapsed) * 60.0)
    end
end

my_new_constraint (generic function with 1 method)

To add this constraint, we can create a new problem function which calls an existing problem and then adds the new constraint to it.

In [29]:
function build_problem_with_added_constraint(pm::ONM.AbstractUnbalancedPowerModel)
    ONM.build_mn_block_mld(pm)

    network_ids = sort(collect(ONM.nw_ids(pm)))
    n_1 = network_ids[1]

    for i in ONM.ids(pm, n_1, :gen)
        my_new_start_constraint(pm, n_1, i)
    end

    for n_2 in network_ids[2:end]
        for i in ONM.ids(pm, n_2, :gen)
            my_new_constraint(pm, n_1, n_2, i)
        end
        n_1 = n_2
    end
end

build_problem_with_added_constraint (generic function with 1 method)

To solve the custom problem:

In [30]:
new_constraint_result = ONM.solve_onm_model(mn_eng_s, ONM.LPUBFDiagPowerModel, solver, build_problem_with_added_constraint; eng2math_passthrough=my_eng2math_passthrough, multinetwork=true)

Dict{String, Any} with 9 entries:
  "solve_time"         => 6.27187
  "optimizer"          => "HiGHS"
  "termination_status" => OPTIMAL
  "dual_status"        => NO_SOLUTION
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 5.34482
  "solution"           => Dict{String, Any}("nw"=>Dict{String, Dict{String, Any…
  "mip_gap"            => 0.0
  "objective_lb"       => 5.34482

We now see that the power output from the generator of interest ramps incrementally

In [31]:
[sum(new_constraint_result["solution"]["nw"]["$n"]["generator"]["675"]["pg"]) for n in sort(parse.(Int,collect(keys(new_constraint_result["solution"]["nw"]))))]

8-element Vector{Float64}:
  90.0
 180.0
 270.0
 360.0
 449.99999999999994
 500.0
 500.0
 500.0

Whereas, previously the power output was always at the maximum

In [32]:
[sum(result["solution"]["nw"]["$n"]["generator"]["675"]["pg"]) for n in sort(parse.(Int,collect(keys(result["solution"]["nw"]))))]

8-element Vector{Float64}:
 500.0
 394.0014720780217
 500.0
 500.0
 500.0
 500.0
 500.0
 500.0

# Adding a Custom Variable

In [33]:
function variable_slack_bus(pm::ONM.AbstractUnbalancedPowerModel; nw::Int=ONM.nw_id_default)
    slack_bus = ONM.var(pm, nw)[:slack_bus] = JuMP.@variable(pm.model, [i in ONM.ids(pm, nw, :bus)], base_name="$(nw)_slack_bus_$(i)", binary=true)

    IM.sol_component_value(pm, PMD.pmd_it_sym, nw, :bus, :slack_bus, ONM.ids(pm, nw, :bus), slack_bus)
end

variable_slack_bus (generic function with 1 method)

In [35]:
function constraint_slack_bus(pm; nw=ONM.nw_id_default)
    for i in ONM.ids(pm, nw, :bus)
        JuMP.@constraint(pm.model, sum(ONM.var(pm, nw, :slack_bus)) == 1)
    end
end

constraint_slack_bus (generic function with 1 method)

In [37]:
function build_problem_with_new_variable(pm)
    ONM.build_mn_block_mld(pm)

    for n in ONM.nw_ids(pm)
        variable_slack_bus(pm; nw=n)
        constraint_slack_bus(pm; nw=n)
    end
end

build_problem_with_new_variable (generic function with 1 method)

In [38]:
new_variable_result = ONM.solve_onm_model(mn_eng_s, ONM.LPUBFDiagPowerModel, solver, build_problem_with_new_variable; multinetwork=true)

Dict{String, Any} with 9 entries:
  "solve_time"         => 6.92859
  "optimizer"          => "HiGHS"
  "termination_status" => OPTIMAL
  "dual_status"        => NO_SOLUTION
  "primal_status"      => FEASIBLE_POINT
  "objective"          => 5.31939
  "solution"           => Dict{String, Any}("nw"=>Dict{String, Dict{String, Any…
  "mip_gap"            => 0.0
  "objective_lb"       => 5.31939

In [39]:
Dict(i => bus["slack_bus"] for (i, bus) in new_variable_result["solution"]["nw"]["1"]["bus"])

Dict{String, Float64} with 25 entries:
  "800aux"    => 0.0
  "671"       => 0.0
  "800"       => 0.0
  "680"       => 0.0
  "634"       => 0.0
  "652"       => 0.0
  "701"       => 0.0
  "675"       => 0.0
  "702"       => 0.0
  "650"       => 0.0
  "600"       => 0.0
  "700"       => 0.0
  "rg60"      => 0.0
  "801"       => 0.0
  "611"       => 0.0
  "645"       => 0.0
  "632"       => 0.0
  "675aux"    => 1.0
  "703"       => 0.0
  "633"       => 0.0
  "684"       => 0.0
  "sourcebus" => 0.0
  "692"       => 0.0
  "670"       => 0.0
  "646"       => 0.0

# Objective Functions

In order to create a new objective function, we merely overwrite the previous one. In this example, let's say that we want to minimize the amount of reactive power at the substation (voltage source).

In [40]:
function my_new_objective(pm::ONM.AbstractUnbalancedPowerModel)
    # voltage source is gen 4
    # voltage sources don't have power limits, use some arbitrarily large ones
    qg_lb, qg_ub = -1e5, 1e5

    qg_sqr = Dict()
    for (n, nw_ref) in ONM.nws(pm)
        qg_sqr[n] = [JuMP.@variable(pm.model, base_name="$(n)_qg_sqr_4_$(c)") for c in ONM.ref(pm, n, :gen, 4, "connections")]
        for (idx,c) in enumerate(ONM.ref(pm, n, :gen, 4, "connections"))
            JuMP.set_lower_bound(ONM.var(pm, n, :qg, 4)[idx], qg_lb)
            JuMP.set_upper_bound(ONM.var(pm, n, :qg, 4)[idx], qg_ub)

            IM.relaxation_product(pm.model, ONM.var(pm, n, :qg, 4)[idx], ONM.var(pm, n, :qg, 4)[idx], qg_sqr[n][idx])
        end
    end

    JuMP.@objective(pm.model, Min, sum( sum(qg_sqr[n])+sum((1-ONM.var(pm, n, :z_block, i)) for (i,block) in nw_ref[:blocks]) for (n, nw_ref) in ONM.nws(pm)))
end

my_new_objective (generic function with 1 method)

We modify the problem function in the same way as before, creating a new function, calling the `build_mn_block_mld` function from ONM, and calling our new function that overwrites the objective.

In [41]:
function build_problem_with_new_objective(pm::ONM.AbstractUnbalancedPowerModel)
    ONM.build_mn_block_mld(pm)

    my_new_objective(pm)
end

build_problem_with_new_objective (generic function with 1 method)

In [42]:
new_objective_result = ONM.solve_onm_model(mn_eng_s, ONM.LPUBFDiagPowerModel, solver, build_problem_with_new_objective; multinetwork=true)

Dict{String, Any} with 9 entries:
  "solve_time"         => 8.15234
  "optimizer"          => "HiGHS"
  "termination_status" => OPTIMAL
  "dual_status"        => NO_SOLUTION
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -2.39998e11
  "solution"           => Dict{String, Any}("nw"=>Dict{String, Dict{String, Any…
  "mip_gap"            => 6.74821e-6
  "objective_lb"       => -2.4e11

Now we can check the reactive power at the substation.

In [43]:
[sum(new_objective_result["solution"]["nw"]["$n"]["voltage_source"]["source"]["qg"]) for n in sort(parse.(Int,collect(keys(new_objective_result["solution"]["nw"]))))]

8-element Vector{Float64}:
  230.41050604169695
  226.26228592508318
 1157.957316562829
 1335.483434025467
 1259.8473186910396
 1325.2366806906377
 1220.6729041517285
 1341.8733176391056

Whereas before it was.

In [44]:
[sum(result["solution"]["nw"]["$n"]["voltage_source"]["source"]["qg"]) for n in sort(parse.(Int,collect(keys(result["solution"]["nw"]))))]

8-element Vector{Float64}:
  987.789854172743
  920.2997773939512
 1000.0
  839.7426189737266
 1607.7887902649272
  687.320201862733
 1030.3336744731614
 1370.7545462577225

Let's compare real power

In [45]:
[sum(new_objective_result["solution"]["nw"]["$n"]["voltage_source"]["source"]["pg"]) for n in sort(parse.(Int,collect(keys(new_objective_result["solution"]["nw"]))))]

8-element Vector{Float64}:
 2321.9390217792125
 2361.0888092681753
 2322.2720672533314
 2402.2720672533314
 2324.272067253331
 2426.772067253331
 2413.2720672533314
 2454.772067253331

In [46]:
[sum(result["solution"]["nw"]["$n"]["voltage_source"]["source"]["pg"]) for n in sort(parse.(Int,collect(keys(result["solution"]["nw"]))))]

8-element Vector{Float64}:
 2711.1018742997467
 3000.0
 2877.54527769662
 2999.999999999999
 3000.000000000001
 3000.000000000001
 3000.0000000000005
 3000.000000000001

# Building Completely Custom Problems

In [47]:
function my_custom_problem(pm::ONM.AbstractUnbalancedPowerModel)
    ONM.variable_block_indicator(pm)

    PMD.variable_mc_bus_voltage_on_off(pm)

    PMD.variable_mc_branch_current(pm)
    PMD.variable_mc_branch_power(pm)

    PMD.variable_mc_switch_power(pm)
    ONM.variable_switch_state(pm)

    PMD.variable_mc_transformer_power(pm)
    PMD.variable_mc_oltc_transformer_tap(pm)

    PMD.variable_mc_generator_power_on_off(pm)

    ONM.variable_mc_storage_power_mi_on_off(pm)

    PMD.variable_mc_load_power(pm)

    PMD.variable_mc_capcontrol(pm)

    PMD.constraint_mc_model_current(pm)

    for i in ONM.ids(pm, :ref_buses)
        PMD.constraint_mc_theta_ref(pm, i)
    end

    ONM.constraint_mc_bus_voltage_block_on_off(pm)

    for i in ONM.ids(pm, :gen)
        ONM.constraint_mc_generator_power_block_on_off(pm, i)
    end

    for i in ONM.ids(pm, :load)
        ONM.constraint_mc_load_power_block_on_off(pm, i)
    end

    for i in ONM.ids(pm, :bus)
        ONM.constraint_mc_power_balance_shed_block(pm, i)
    end

    for i in ONM.ids(pm, :storage)
        PMD.constraint_storage_state(pm, i)
        ONM.constraint_storage_complementarity_mi_block_on_off(pm, i)
        ONM.constraint_mc_storage_block_on_off(pm, i)
        ONM.constraint_mc_storage_losses_block_on_off(pm, i)
        PMD.constraint_mc_storage_thermal_limit(pm, i)
    end

    for i in ONM.ids(pm, :branch)
        PMD.constraint_mc_power_losses(pm, i)
        PMD.constraint_mc_model_voltage_magnitude_difference(pm, i)
        PMD.constraint_mc_voltage_angle_difference(pm, i)

        PMD.constraint_mc_thermal_limit_from(pm, i)
        PMD.constraint_mc_thermal_limit_to(pm, i)

        PMD.constraint_mc_ampacity_from(pm, i)
        PMD.constraint_mc_ampacity_to(pm, i)
    end

    ONM.constraint_isolate_block(pm)
    for i in ONM.ids(pm, :switch)
        ONM.constraint_mc_switch_state_open_close(pm, i)

        PMD.constraint_mc_switch_thermal_limit(pm, i)
        PMD.constraint_mc_switch_ampacity(pm, i)
    end

    for i in ONM.ids(pm, :transformer)
        ONM.constraint_mc_transformer_power_block_on_off(pm, i; fix_taps=false)
    end

    my_new_objective(pm)
end

my_custom_problem (generic function with 1 method)

In [48]:
my_problem_result = ONM.solve_onm_model(eng_s, ONM.LPUBFDiagPowerModel, solver, my_custom_problem)

Dict{String, Any} with 9 entries:
  "solve_time"         => 0.0259009
  "optimizer"          => "HiGHS"
  "termination_status" => OPTIMAL
  "dual_status"        => NO_SOLUTION
  "primal_status"      => FEASIBLE_POINT
  "objective"          => -3.0e10
  "solution"           => Dict{String, Any}("voltage_source"=>Dict{String, Any}…
  "mip_gap"            => 2.3307e-10
  "objective_lb"       => -3.0e10